# Tarea N°2 - Máquinas de Aprendizaje - ILI393
### Martín Villanueva A.

## Introducción

## Parte 1 - Evaluación de Créditos

### Linear Discriminant Analysis

In [16]:
solve_clf('LDA', 'credit', data_func=normalize)

##############################################################
Dataset: 0
Training error: 0.0777777777778
Testing error: 0.133333333333
##############################################################


##############################################################
Dataset: 1
Training error: 0.0555555555556
Testing error: 0.1
##############################################################


##############################################################
Dataset: 2
Training error: 0.0222222222222
Testing error: 0.133333333333
##############################################################


##############################################################
Dataset: 3
Training error: 0.0555555555556
Testing error: 0.166666666667
##############################################################


##############################################################
Dataset: 4
Training error: 0.0555555555556
Testing error: 0.1
##############################################################


#################

### Naive Bayes

In [245]:
params = {'nbins':np.array([2,4,6,8,10])}

In [ ]:
solve

### Primal Perceptron

In [27]:
#parameters to try on primal perceptron
params = {'eta':np.linspace(1e0, 1e2, 5, endpoint=True)}

In [28]:
solve_clf('PP', 'credit', params=params, data_func=normalize)

##############################################################
Dataset: 0
Best parameter: {'eta': 1.0}
CV scores:
mean: 0.91111, std: 0.05554, params: {'eta': 1.0}
mean: 0.91111, std: 0.05554, params: {'eta': 25.75}
mean: 0.91111, std: 0.05554, params: {'eta': 50.5}
mean: 0.91111, std: 0.05554, params: {'eta': 75.25}
mean: 0.91111, std: 0.05554, params: {'eta': 100.0}
Training error: 0.0
Testing error: 0.1
##############################################################


##############################################################
Dataset: 1
Best parameter: {'eta': 1.0}
CV scores:
mean: 0.92222, std: 0.07565, params: {'eta': 1.0}
mean: 0.92222, std: 0.07565, params: {'eta': 25.75}
mean: 0.92222, std: 0.07565, params: {'eta': 50.5}
mean: 0.92222, std: 0.07565, params: {'eta': 75.25}
mean: 0.92222, std: 0.07565, params: {'eta': 100.0}
Training error: 0.0
Testing error: 0.0666666666667
##############################################################


######################################

### Nu-SVM with Linear Kernel

In [34]:
#parameters to try on Nu-SVM with linear kernel
params = {'nu':np.array([0.1, 0.2, 0.3, 0.4, 0.5])}

In [35]:
solve_clf('lSVM', 'credit', params=params, data_func=normalize)

##############################################################
Dataset: 0
Best parameter: {'nu': 0.20000000000000001}
CV scores:
mean: 0.93333, std: 0.04175, params: {'nu': 0.10000000000000001}
mean: 0.94444, std: 0.03340, params: {'nu': 0.20000000000000001}
mean: 0.90000, std: 0.03931, params: {'nu': 0.29999999999999999}
mean: 0.88889, std: 0.03374, params: {'nu': 0.40000000000000002}
mean: 0.87778, std: 0.04289, params: {'nu': 0.5}
Training error: 0.0333333333333
Testing error: 0.1
##############################################################


##############################################################
Dataset: 1
Best parameter: {'nu': 0.10000000000000001}
CV scores:
mean: 0.95556, std: 0.04376, params: {'nu': 0.10000000000000001}
mean: 0.94444, std: 0.05001, params: {'nu': 0.20000000000000001}
mean: 0.93333, std: 0.05580, params: {'nu': 0.29999999999999999}
mean: 0.92222, std: 0.04607, params: {'nu': 0.40000000000000002}
mean: 0.92222, std: 0.06952, params: {'nu': 0.5}
Training

## Parte 2 - Pima Indians Diabetes

### Naive Bayes

### Dual Perceptron with Gaussian Kernel

In [36]:
params = {'eta':np.linspace(1.,10.,5,endpoint=True), 'gamma':np.linspace(0.5,2.,5,endpoint=True)}

In [37]:
solve_clf('DP', 'credit', params=params, data_func=normalize)

##############################################################
Dataset: 0
Best parameter: {'eta': 1.0, 'gamma': 1.25}
CV scores:
mean: 0.86667, std: 0.04921, params: {'eta': 1.0, 'gamma': 0.5}
mean: 0.85556, std: 0.06007, params: {'eta': 1.0, 'gamma': 0.875}
mean: 0.88889, std: 0.05204, params: {'eta': 1.0, 'gamma': 1.25}
mean: 0.87778, std: 0.04362, params: {'eta': 1.0, 'gamma': 1.625}
mean: 0.86667, std: 0.03095, params: {'eta': 1.0, 'gamma': 2.0}
mean: 0.86667, std: 0.04921, params: {'eta': 3.25, 'gamma': 0.5}
mean: 0.85556, std: 0.06007, params: {'eta': 3.25, 'gamma': 0.875}
mean: 0.88889, std: 0.05204, params: {'eta': 3.25, 'gamma': 1.25}
mean: 0.87778, std: 0.04362, params: {'eta': 3.25, 'gamma': 1.625}
mean: 0.86667, std: 0.03095, params: {'eta': 3.25, 'gamma': 2.0}
mean: 0.86667, std: 0.04921, params: {'eta': 5.5, 'gamma': 0.5}
mean: 0.85556, std: 0.06007, params: {'eta': 5.5, 'gamma': 0.875}
mean: 0.88889, std: 0.05204, params: {'eta': 5.5, 'gamma': 1.25}
mean: 0.87778, std: 0

### Nu-SVM with Gaussian Kernel

In [47]:
params = {'nu':np.array([0.1, 0.2, 0.3, 0.4, 0.5]), 'gamma':np.linspace(0.5,2.,5,endpoint=True)}

In [48]:
solve_clf('kSVM', 'credit', params=params, data_func=normalize)

##############################################################
Dataset: 0
Best parameter: {'nu': 0.5, 'gamma': 2.0}
CV scores:
mean: 0.87778, std: 0.07209, params: {'nu': 0.10000000000000001, 'gamma': 0.5}
mean: 0.87778, std: 0.07209, params: {'nu': 0.20000000000000001, 'gamma': 0.5}
mean: 0.86667, std: 0.02825, params: {'nu': 0.29999999999999999, 'gamma': 0.5}
mean: 0.88889, std: 0.03864, params: {'nu': 0.40000000000000002, 'gamma': 0.5}
mean: 0.88889, std: 0.03939, params: {'nu': 0.5, 'gamma': 0.5}
mean: 0.87778, std: 0.07209, params: {'nu': 0.10000000000000001, 'gamma': 0.875}
mean: 0.87778, std: 0.07209, params: {'nu': 0.20000000000000001, 'gamma': 0.875}
mean: 0.86667, std: 0.05459, params: {'nu': 0.29999999999999999, 'gamma': 0.875}
mean: 0.86667, std: 0.05459, params: {'nu': 0.40000000000000002, 'gamma': 0.875}
mean: 0.87778, std: 0.04235, params: {'nu': 0.5, 'gamma': 0.875}
mean: 0.88889, std: 0.05100, params: {'nu': 0.10000000000000001, 'gamma': 1.25}
mean: 0.88889, std: 0.051

## Conclusiones

## Anexo

### Configuración del notebook

In [1]:
#notebook settings
%matplotlib inline

#import some useful libraries and utilities
import numba
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cross_validation import KFold
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.grid_search import GridSearchCV

#setting some paths
path1 = './credit/'
#data directory
path2 = './diabetes/'

### Funciones para el manejo de la data

In [2]:
#Rescale features of M to [a,b] range
def rescale(M, a=0., b=1.):
    #max and min vectors
    maxv = np.max(M, axis=0)
    minv = np.min(M, axis=0)
    return (b-a)*M/(maxv-minv) + (a*maxv-b*minv)/(maxv-minv)

#Normalize features of M
def normalize(M):
    #mean and standard deviation vectors
    meanv = np.mean(M, axis=0)
    stdv = np.std(M, axis=0)
    return (M-meanv)/stdv

## Métricas de error para clasificación

In [3]:
#precision
def precision(yreal, ypred):
    yr = yreal.astype(int)
    yp = ypred.astype(int)
    m, = yr.shape
    #number of good predictions/number of predictions
    return np.sum(yp==yr)/np.float(m)

#error rate 
def error_rate(yreal, ypred):
    return 1.-precision(yreal, ypred)

## Implementación de LDA

In [4]:
class LDA(BaseEstimator,ClassifierMixin):
    def __init_(self):
        pass
    
    def fit(self, X, y):
        #boolean masks
        mask1 = y.astype(bool)
        mask0 = np.logical_not(mask1)
        #number of samples in each class
        M,_ = X.shape
        M0 = np.sum(mask0)
        M1 = M-M0
        #estimates of probabilities
        p0 = M0/np.float(M)
        p1 = 1-p0
        #estimates of means
        u0 = X[mask0].sum(axis=0)/M0
        u1 = X[mask1].sum(axis=0)/M1
        #estimation of shared covariance matrix
        Y = X.copy()
        Y[mask0] -= u0
        Y[mask1] -= u1
        Cov = np.dot(Y.T,Y)
        Cov /= (M-2.)
        #computing parameters of linear discriminants
        self.w0 = np.linalg.solve(Cov,u0)
        self.b0 = np.log(p0) - 0.5*np.dot(u0, self.w0)
        self.w1 = np.linalg.solve(Cov,u1)
        self.b1 = np.log(p1) - 0.5*np.dot(u1, self.w1)
    
    def predict(self, X):
        #evaluation of discriminant hyperplane
        return (np.dot(X,self.w1-self.w0) > self.b0-self.b1)

## Implementación de Naive Bayes

In [253]:
class NB(BaseEstimator,ClassifierMixin):
    def __init__(self, disc_type=None, nbins=5, features=None):
        self.disc_type = disc_type
        self.nbins = nbins
        self.features = features
        
        
    #equal width discretization
    def EWD(self, x):
        eps = 1e-10
        bins =  np.linspace(x.min()-eps, x.max()+eps, self.nbins+1, endpoint=True)
        return bins
    
    #equal frequency discretization
    def EFD(self, x):
        eps = 1e-10
        bins = np.empty(self.nbins+1)
        
        return
    
    def fit(self, X, y):
        #boolean masks
        mask1 = y.astype(bool)
        mask0 = np.logical_not(mask1)
        #number of samples in each class
        M,N = X.shape
        M0 = np.sum(mask0)
        M1 = M-M0
        #estimates of probabilities
        p0 = M0/np.float(M)
        p1 = 1-p0
        #separating data by classes
        X0 = X[mask0]
        X1 = X[mask1]
        #conditional probabilities of features in each class
        CP0 = list()
        CP1 = list()
        #matrix of bins for each non-discrete feature
        B0 = np.empty((len(self.features),self.nbins+1))
        B1 = np.empty((len(self.features),self.nbins+1))
        ind = 0
        
        #tolerance on interval extremes of discretization
        eps = 1e-10
        
        #computing conditional probabilities of features
        for f in xrange(N):
            #discretization needed
            if f in self.features:
                f0 = X0[:,f]
                f1 = X1[:,f]
                bins0 = self.EWD(f0)
                bins1 = self.EWD(f1)
                frec0 = np.bincount(np.digitize(f0, bins0), minlength=self.nbins+2) + 1.
                frec1 = np.bincount(np.digitize(f1, bins1), minlength=self.nbins+2) + 1.
                CP0.append(frec0/(M0+self.nbins+2))
                CP1.append(frec1/(M1+self.nbins+2))
                B0[ind] = bins0
                B1[ind] = bins1
                ind += 1
            #no discretization needed
            else:
                f0 = X0[:,f].astype(int)
                f1 = X1[:,f].astype(int)
                frec0 = np.bincount(f0, minlength=np.max(f0)+1) + 1.
                frec1 = np.bincount(f1, minlength=np.max(f1)+1) + 1.
                CP0.append(frec0/(M0+np.max(f0)+1))
                CP1.append(frec1/(M1+np.max(f1)+1))
                
        #storing needed data
        self.p0 = p0
        self.p1 = p1 
        self.CP0 = CP0
        self.CP1 = CP1
        self.B0 = B0
        self.B1 = B1
        return self
        
    def predict(self, X):
        M,N = X.shape
        prob0 = np.ones(M)
        prob1 = np.ones(M)
        ind = 0
        for f in xrange(N):
            #discretization needed
            if f in self.features:
                tmp = X[:,f]
                tmp0 = np.digitize(tmp, self.B0[ind]).astype(int)
                tmp1 = np.digitize(tmp, self.B1[ind]).astype(int)
                prob0 *= (self.CP0[f])[tmp0]
                prob1 *= (self.CP1[f])[tmp1]
                ind +=1
            #no discretization needed
            else:
                tmp = X[:,f].astype(int)
                prob0 *= (self.CP0[f])[tmp]
                prob1 *= (self.CP1[f])[tmp]
        prob0 *= self.p0
        prob1 *= self.p1
        return (prob0 < prob1).astype(int)
    
    #to make it compatible with scikit
    def score(self, X, y):
        return precision(y, self.predict(X))

In [251]:
wa = np.load(path1+'credit-tr-10.npy')
X = wa[:,:-1]
y = wa[:,-1]

In [254]:
nb = NB(features=[0,1,2,3,5], nbins=10)
nb.fit(X,y)

NB(disc_type=None, features=[0, 1, 2, 3, 5], nbins=10)

In [255]:
error_rate(y,nb.predict(X))

0.05555555555555558

## Implementación de Primal Perceptron

In [6]:
class PP(BaseEstimator,ClassifierMixin):
    def __init__(self, eta=0.01, n_iter=100000, tol=25):
        self.eta = eta
        self.n_iter = n_iter
        self.tol = tol
    
    def f(self, X):
        return np.dot(X,self.w)+self.b
    
    def fit(self, X, y):
        M,N = X.shape
        #initial guesses
        self.w = np.zeros(N)
        self.b = 0.
        #R parameter
        R = np.max(np.linalg.norm(X, axis=1))
        #error rate and counter
        err1 = error_rate(y, self.predict(X))
        c = 0
        
        for i in xrange(self.n_iter):
            #error rate at previous iteration
            err0 = err1
            for m in xrange(M):
                #if misclassified
                if y[m]*self.f(X[m])<=0:
                    self.w += self.eta*y[m]*X[m]
                    self.b += self.eta*y[m]*R**2
            err1 = error_rate(y, self.predict(X))
            #stopping criterions
            if err1==0.: break
            elif err0==err1:
                c += 1
                if c > self.tol: break
            else: c = 0
        #store the number of iterations needed
        self.it = i+1
        return self
        
    def predict(self, X):
        return np.sign(self.f(X))
    
    #to make it compatible with scikit
    def score(self, X, y):
        return precision(y, self.predict(X))

+ La primera vez que se implemento, en la regla de actualización para $b$ se puso equívocamente: $b \leftarrow b + \nu y_m R \ $    en vez de $R^2$, y los resultados obtenidos fueron mucho mejores.

## Implementación de Dual Perceptron

In [10]:
class DP(BaseEstimator,ClassifierMixin):
    def __init__(self, eta=0.01, gamma=0.5, n_iter=100000, tol=20):
        self.eta = eta
        self.gamma = gamma
        self.n_iter = n_iter
        self.tol = tol
        pass

    numba.jit()
    def fit(self, X, y):
        M,N = X.shape
        #store data
        self.X = X
        self.y = y
        #initial guesses
        self.alp = np.zeros(M)
        self.b = 0.
        #R parameter
        R = np.max(np.linalg.norm(X, axis=1))
        #precomputed kernel matrix
        self.K = rbf_kernel(X, gamma=self.gamma)
        #error rate and counter
        err1 = error_rate(y, self.predict(X))
        c = 0
        
        for i in xrange(self.n_iter):
            #error rate at previous iteration
            err0 = err1
            for m in xrange(M):
                #if misclassified
                if y[m]*(np.sum(self.alp*y*self.K[m])+self.b)<=0:
                    self.alp[m] += 1
                    self.b += self.eta*y[m]*R**2
            err1 = error_rate(y, self.predict(X))
            #stopping criterions
            if err1==0.: break
            elif err0==err1:
                c += 1
                if c > self.tol: break
            else: c = 0
        #store the number of iterations needed
        self.it = i+1
        return self
        
    def predict(self, X):
        if np.array_equal(X, self.X):
            K = self.K
        else:
            #compute the kernel matrix
            K = rbf_kernel(X, self.X, self.gamma)
        f = np.sum(self.alp*self.y*K, axis=1)+self.b
        return np.sign(f)
    
    #to make it compatible with GridSearch
    def score(self, X, y):
        return precision(y, self.predict(X))

## Funciones complementarias (helpers) para obtener resultados

In [46]:
"""
Function to generate histogram of winners
"""
def make_hist(winners,params):
    winners = np.array(winners)
    freqs = np.zeros(5)
    for i in xrange(5):
        freqs[i] = np.sum(params[i]==winners)

    labels = map(str,params)
    pos = np.arange(len(labels))
    width = 1.0
    fig = plt.figure()
    fig.set_figheight(5)
    fig.set_figwidth(5)
    ax = plt.axes()
    ax.set_xticks(pos + (width / 2))
    ax.set_xticklabels(labels)
    plt.ylabel('Frequency')
    plt.title('Histogram of winner parameters')
    plt.bar(pos, freqs, width, color='0.5')
    plt.show()

"""
Generate solutions for classification problems
"""
def solve_clf(algorithm, dataset, params=None, grid=None, data_func=None, show=None):
    #if params is not None:
    #    winners = list()
        
    #set dataset name
    if dataset=='credit':
        path = './credit/credit'
    elif dataset=='diabetes':
        path = './diabetes/diabetes'
    else: return -1
    
    #set classifier
    if algorithm=='LDA':   clf = LDA()
    elif algorithm=='NB':  clf = NB()
    elif algorithm=='PP':  clf = PP()
    elif algorithm=='DP':  clf = DP()
    elif algorithm=='lSVM': clf = svm.NuSVC(kernel='linear')
    elif algorithm=='kSVM': clf = svm.NuSVC(kernel='rbf')
    else: return -1
    
    #iterate through data
    for i in xrange(20):
        #loading dataset
        tr_file = path+'-tr-{0}.npy'.format(i)
        ts_file = path+'-ts-{0}.npy'.format(i)
        tr_data = np.load(tr_file)
        ts_data = np.load(ts_file)
        
        if data_func is not None:
            X_tr = data_func(tr_data[:,:-1])
            X_ts = data_func(ts_data[:,:-1])
        else: 
            X_tr = tr_data[:,:-1]
            X_ts = ts_data[:,:-1]
        y_tr = np.ascontiguousarray(tr_data[:,-1])
        y_ts = np.ascontiguousarray(ts_data[:,-1])
        if algorithm in ['PP', 'DP', 'SVM']:
            #change label for separating hyperplanes approaches
            y_tr[y_tr==0.] = -1
            y_ts[y_ts==0. ]= -1

        if params is not None:
            #tunning parameters
            gs = GridSearchCV(clf, params, cv=5, n_jobs=4)
            gs.fit(X_tr, y_tr)
            grid_scores = gs.grid_scores_
            best_params = gs.best_params_
            #train the data 
            clf.set_params(**best_params)
            clf.fit(X_tr, y_tr)
        else:
            clf.fit(X_tr, y_tr)
        
        if (show is not None) and (i not in show): continue
        print "##############################################################"
        print "Dataset: {0}".format(i)
        if params is not None:
            print 'Best parameter: {0}'.format(best_params)
            print 'CV scores:'
            for result in grid_scores: print result
        print 'Training error: {0}'.format(error_rate(y_tr, clf.predict(X_tr)))
        print 'Testing error: {0}'.format(error_rate(y_ts, clf.predict(X_ts)))
        print "##############################################################"
        print '\n'
    
    #if params is not None:
        #make_hist(winners,params)